<a href="https://colab.research.google.com/github/ashikshafi08/Learning-Fastai/blob/main/AI_Crowd_NLP_Blitz/Fastai_Approach_Research_Paper_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai --upgrade

In [2]:
!pip install -q -U aicrowd-cli
API_KEY = '' 
!aicrowd login --api-key $API_KEY

     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 174kB 13.1MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 215kB 16.7MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
API Key valid
Saved API Key successfully!


In [3]:
# Downloading the Dataset ( removing data and assets folder if existing already and then creating the folder )
!rm -rf data
!mkdir data
!rm -rf assets
!mkdir assets

!aicrowd dataset download --challenge research-paper-classification -j 3 -o data # Downloading the dataset and saving it in data folder

val.csv:   0% 0.00/883k [00:00<?, ?B/s]
test.csv:   0% 0.00/3.01M [00:00<?, ?B/s]

val.csv: 100% 883k/883k [00:00<00:00, 6.44MB/s]

test.csv: 100% 3.01M/3.01M [00:00<00:00, 15.4MB/s]


train.csv: 100% 8.77M/8.77M [00:00<00:00, 34.9MB/s]


In [4]:
# Importing the needed packages
import fastai
from fastai import *
from fastai.text.all import * 
import pandas as pd 
print(fastai.__version__)


2.3.1


In [5]:
# Importing the data 

train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/val.csv')
test_data = pd.read_csv('data/test.csv')

# Printing out all shapes of our data 
print(f'Shape of the train data: {train_data.shape}')
print(f'Shape of the validation data: {val_data.shape}')
print(f'Shape of the test data: {test_data.shape}')

Shape of the train data: (31500, 3)
Shape of the validation data: (2700, 3)
Shape of the test data: (10800, 3)


In [6]:
data = pd.concat([train_data , val_data] , axis = 0)

In [7]:
data.isna().sum()

id       0
text     0
label    0
dtype: int64

In [8]:
# Creating a DataBlock 
db = DataBlock(blocks = (TextBlock.from_df('text') , CategoryBlock) , 
               get_x = ColReader('text') , 
               get_y = ColReader('label') , 
               splitter = RandomSplitter(0.1))

In [9]:
# DataLoaders 
dls = db.dataloaders(data , bs =32)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [10]:
dls.show_batch()

,text,category
0,"xxbos despite over the years its main focus has been the support of processes in highly controlled domains , many domains of interest to the xxup xxunk community are characterized by ever - changing requirements , unpredictable environments and increasing amounts of data that influence the execution of process instances . xxup ai has concentrated its efforts on investigating dynamic domains that involve active control of computational entities and physical devices ( e.g. , robots , software agents , etc . ) in this context , xxmaj automated xxmaj planning , which is one of the oldest areas in xxup ai , is conceived as",0
1,"xxbos in the first optimization , we compute the xxmaj center of xxmaj mass ( com ) trajectory , foot step locations , and introduce xxunk variables to account for violating the imposed constraints on the xxmaj zero xxmaj moment xxmaj point ( zmp ) the second optimization , in which we calculate the optimal external force that compensates for the xxup zmp tracking error . once contact is created , the xxup xxunk reduces to a single xxmaj quadratic xxmaj program ( qp ) that can be solved in real - time ( $ $ xxunk ) .",2
2,"xxbos . xxmaj class imbalance is a common problem in real - world object detection and classification tasks . xxmaj data of some classes is abundant making them an over - represented majority , and data of others is scarce . xxmaj this imbalance makes it challenging for classifiers to appropriately learn the discriminating boundaries of the majority and minority classes . xxmaj the proposed approach is applicable to both binary and multi - class problems without any modification . xxmaj moreover , as opposed to data level approaches , we do not alter the original data distribution",3
3,"xxbos and relation prediction , respectively . xxmaj to achieve these goals , we propose a novel recurrent neural network model , called xxmaj hierarchical long - term xxmaj memory ( h - lstm ) . xxmaj it contains two coupled sub - networks : the xxmaj pixel ( p - ltm ) , and the xxmaj multi - scale xxmaj super - pixel ( ms - xxunk ) . xxmaj the model is capable of parsing scene geometric structures and outperforming several state - of - the - art methods by large margins",3
4,"xxbos to learn a causal structure via interaction and learning . xxmaj we test our method over two different scenarios , and the results presented for the first test scenario demonstrate the usefulness of our technique to learning an effective action . xxmaj on the other hand , the second experiment , shows that our proposal manages to understand the underlying cause of several tasks with different sizes and different causal structures . a xxmaj random xxmaj graph is an random object which take its values in the space of graphs .",0
5,"xxbos the emergence or bootstrap of a lexicon involves the repeated interaction between at least two agents who must reach a consensus on how to name n objects using h words . here we consider minimal models of two types of learning algorithms : cross - situational learning , in which the individuals determine the meaning of the word by looking for something in common across all observed uses of that word . we show that they yield the same communication accuracy in the realistic limits of large n and h .",1
6,"xxbos and the post - classification pipeline perform classification on each video snippet and then aggregate the classification scores to obtain the video - level classification score . xxmaj moreover , we discover that an ideal classifier may possess two characteristics : 1 ) xxmaj the frame level classification results obtained from the pre - classification stream should be consistent ; 3 ) xxmaj the class results of these two streams should not be identical . xxmaj this achieves the novel xxmaj equivalent xxmaj classification xxmaj mapping ( xxunk ) mechanism .",3
7,"xxbos video - based person re - identification ( re - id ) aims at matching the same person across video clip

In [25]:
f1_score = F1Score(average = 'weighted')

In [73]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy , f1_score] , drop_mult= 0.4 , wd = 0.4 ,seq_len= 128)
learn

In [74]:
learn.fine_tune(10 , 2e-2 , lr_mult = 125 , pct_start= 0.4)

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.834396,0.767437,0.706140,0.704514,00:47


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.532614,0.550512,0.786550,0.790050,01:24
1,0.487497,0.563081,0.801754,0.794954,01:24
2,0.463845,0.456923,0.833626,0.822568,01:25
3,0.455221,0.448122,0.836257,0.834150,01:24
4,0.392752,0.491956,0.828070,0.826328,01:24
5,0.355973,0.425789,0.855556,0.853493,01:24
6,0.281772,0.449395,0.847368,0.850680,01:24
7,0.233548,0.502182,0.851170,0.843596,01:24
8,0.166698,0.494890,0.852339,0.848096,01:24
9,0.144732,0.508216,0.857895,0.857413,01:24


In [75]:
test_data.drop('label' , inplace = True , axis = 1)
test_data.head()

,id,text
0,0,"we propose a lightweight framework to detect iris presentation attacks by extracting multiple micro-stripes of expanded normalized iris textures . in this procedure, the segmented area is processed to provide lower dimensional input segments and a higher number of learning samples ."
1,1,"the proposed method presents an alternate solution to SLAM for map update in terms of efficiency, cost, availability, and map reuse . a virtual LIDAR that estimates the depth of objects in the 3D map is used to generate a compact point cloud ."
2,2,proposed ear identification method fusing SIFT features of color segmented slice regions of an ear . proposed method makes use of Gaussian mixture model (GMM) to build ear model using vector quantization algorithm and K-L divergence .
3,3,a method to reconstruct the three-dimensional trajectory of a moving instance of an object category using stereo video data . we track the two-dimensional shape of objects on pixel level .
4,4,"strong local consistencies can improve their performance by largely preserving the speed-ups they offer in cases where they are successful, and eliminating run time penalties . both algorithms consist of a master search process, which is a typical CSP solver, and a number of slave processes, with each one implementing a SLC method ."


In [76]:
test_dl = learn.dls.test_dl(test_data)
preds = learn.get_preds(dl=test_dl)

In [77]:
test_data['label'] = preds[0].argmax(dim = 1)
test_data.head()

,id,text,label
0,0,"we propose a lightweight framework to detect iris presentation attacks by extracting multiple micro-stripes of expanded normalized iris textures . in this procedure, the segmented area is processed to provide lower dimensional input segments and a higher number of learning samples .",3
1,1,"the proposed method presents an alternate solution to SLAM for map update in terms of efficiency, cost, availability, and map reuse . a virtual LIDAR that estimates the depth of objects in the 3D map is used to generate a compact point cloud .",2
2,2,proposed ear identification method fusing SIFT features of color segmented slice regions of an ear . proposed method makes use of Gaussian mixture model (GMM) to build ear model using vector quantization algorithm and K-L divergence .,3
3,3,a method to reconstruct the three-dimensional trajectory of a moving instance of an object category using stereo video data . we track the two-dimensional shape of objects on pixel level .,3
4,4,"strong local consistencies can improve their performance by largely preserving the speed-ups they offer in cases where they are successful, and eliminating run time penalties . both algorithms consist of a master search process, which is a typical CSP solver, and a number of slave processes, with each one implementing a SLC method .",0


In [78]:
import os
#!mkdir assets
test_data.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [79]:
!aicrowd notebook submit -c research-paper-classification -a assets --no-verify

Using notebook: /content/drive/MyDrive/Colab Notebooks/Fastai_Approach_Research_Paper_Classification.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
submission.zip ━━━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 1.1/1.1 MB • 1.2 MB/s • 0:00:00
                                                        ╭─────────────────────────╮                                                        
                                                        │ Successfully submitted! │                                                        
                                                        ╰─────────────────────────╯                                                        
                                                              Important links                                                              
┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│